### Importar bibliotecas

In [ ]:
import numpy as np
import pandas as pd
import re as regex

### Carga del dataset desde el archivo

In [ ]:
pathArchivoDataSet = 'properatti.csv'
ds = pd.read_csv(pathArchivoDataSet)

### Conversión de los valores NaN de la columna Description

#### A todos los valores NaN se los setea como string vacío ('')

In [ ]:
ds['description'].fillna('');

### Contamos la cantidad de rows que no tienen un valor en rooms

In [ ]:
ds['rooms'].isna().sum()

### Método que a partir de un text, retorna la cantidad de ambientes

In [ ]:
def getRoomsNumberFromText(text):
    roomsCount = 0;
    result = '';
    
    #para cuando el texto tiene números, ej: '2 ambientes'
    m = regex.search('\d', text)
    
    if(m is not None):
        result = m.group(0)
    
    if(result == ''):
        #no funcionó el parseo por números, se busca por palabras: monoambiente, dos ambientes, tres ambientes
        if('monoambiente' in text.lower()): roomsCount = 1
        if('uno' in text.lower()): roomsCount = 1
        if('dos' in text.lower()): roomsCount = 2
        if('tres' in text.lower()): roomsCount = 3
        if('cuatro' in text.lower()): roomsCount = 4
        if('cinco' in text.lower()): roomsCount = 5
    else:
        roomsCount = int(result)
    
    return roomsCount
    

### Método que a partir de un dato Description del dataset, retorna la cantidad de ambientes

#### Este método llama getRoomsNumberFromText() para obtener el número de ambientes

In [ ]:
def getRoomsFromDescription(description):
    
    #array con expresiones regulares de búsqueda del dato 'cantidad de ambientes'
    #se van a buscar estos string por separado en el parámetro description
    roomsNumberRegexs = ['\d ambientes', 'monoambiente', 'dos ambientes', 'tres ambientes']
    
    roomsNumber = ''
    
    #primer búsqueda, que se nombre en la description la cantidad de ambientes
    #por cada regex en roomsNumberRegexs
    for expr in roomsNumberRegexs:
        #busca si la expresion regular está en description
        m = regex.search(expr, description.lower())
        
        if(m is not None):
            roomsNumber = m.group(0)
            break #como ya se encontró, no se sigue buscando, se sale del ciclo
                    
    #segunda búsqueda, si se encontró el texto con los ambientes, se obtiene el número de ambientes
    if(roomsNumber != ''):
        return getRoomsNumberFromText(description)

### Método para actualizar la cantidad de ambientes, según la description

#### Este método llama a getRoomsFromDescription(), para obtener la cantidad de rooms desde la description y con eso actualizar el dato Rooms

In [ ]:
def updateRoomsQuantityFromDescription(dataSetRow):
    try:
        if(dataSetRow.description is not None):
            
            roomsCount = 0

            roomsCount = getRoomsFromDescription(dataSetRow.description)

            if(roomsCount is not None):
                if(roomsCount > 0):
                    dataSetRow.rooms = roomsCount
        
        return dataSetRow
    except:
        print(dataSetRow[0])

### Ejectuamos la actualización y luego verificamos la cantidad de rows que siguen sin tener dato en la columna Rooms

In [ ]:
ds = ds.apply(updateRoomsQuantityFromDescription, axis=1);

In [ ]:
ds['rooms'].isna().sum()